# Prueba del modelo preentrenado con SSL

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
import torch.optim as optim

# Importa las librerías de MedMNIST
import medmnist
from medmnist import INFO  
import torchvision.transforms as transforms 

# --- 1. Configuración ---
SSL_WEIGHTS_PATH = "models/221025MG_backbone.ssl.pth"
DATASET_NAME = 'breastmnist'
BATCH_SIZE = 128                          
NUM_EPOCHS = 5                            
LEARNING_RATE = 1e-4                      # LR bajo es bueno para finetuning
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Usando dispositivo: {DEVICE}")

# --- 2. Definir el Modelo de Clasificación ---
# Este modelo combina el backbone + una cabeza para clasificar

class SslClassifier(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()
        self.backbone = backbone
        # La salida de ResNet18 (antes de la última capa) es 512
        self.fc = nn.Linear(512, num_classes) # Esta es la nueva cabeza

    def forward(self, x):
        # Pasa por el backbone (que ya sabe de SSL)
        features = self.backbone(x).flatten(start_dim=1)
        # Pasa por la cabeza de clasificación
        output = self.fc(features)
        return output

# --- 3. Cargar Datos (BreastMNIST) ---
info = INFO[DATASET_NAME]
num_classes = len(info['label'])
DataClass = getattr(medmnist, info['python_class'])


mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

data_transform = transforms.Compose([
    # Convierte la imagen (de 1 canal) a 3 canales duplicando el canal.
    transforms.Grayscale(num_output_channels=3),
    
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std) 
])

# Usamos el mismo transform para todos
train_transform = data_transform
val_transform = data_transform
test_transform = data_transform

# Crear los datasets
train_dataset = DataClass(split='train', transform=train_transform, download=True)
val_dataset = DataClass(split='val', transform=val_transform, download=True)
test_dataset = DataClass(split='test', transform=test_transform, download=True)

# Crear los DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Datos de {DATASET_NAME} cargados. Clases: {num_classes}")

# --- 4. Cargar Modelo y Pesos SSL ---

# 1. Crea la arquitectura del backbone (la misma que en SSL)
resnet = models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])

# 2. Carga los pesos que entrenaste con SSL
# Esto carga los pesos en el objeto 'backbone'
backbone.load_state_dict(torch.load(SSL_WEIGHTS_PATH, map_location=DEVICE))
print("¡Pesos del backbone SSL cargados exitosamente!")

# 3. Crea el modelo de clasificación completo
model = SslClassifier(backbone, num_classes).to(DEVICE)

# --- 5. Configurar Entrenamiento (Finetuning) ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"--- Iniciando Finetuning (5 épocas) en {DATASET_NAME} ---")

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        labels = labels.squeeze().long()
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}")

    # Validación después de cada época
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            labels = labels.squeeze().long()
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_acc = 100 * val_correct / val_total
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Val Accuracy: {val_acc:.2f}%")

print("--- Finetuning completado ---")

# --- 6. Prueba Final (Clasificación) ---
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.squeeze().long()
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc = 100 * test_correct / test_total
print(f"==================================================")
print(f"✅ Resultado Final - Test Accuracy: {test_acc:.2f}%")
print(f"==================================================")

Usando dispositivo: cuda
Datos de breastmnist cargados. Clases: 2
¡Pesos del backbone SSL cargados exitosamente!
--- Iniciando Finetuning (5 épocas) en breastmnist ---
Epoch [1/5], Train Loss: 0.7467
Epoch [1/5], Val Accuracy: 74.36%
Epoch [2/5], Train Loss: 0.6739
Epoch [2/5], Val Accuracy: 74.36%
Epoch [3/5], Train Loss: 0.6025
Epoch [3/5], Val Accuracy: 75.64%
Epoch [4/5], Train Loss: 0.5706
Epoch [4/5], Val Accuracy: 73.08%
Epoch [5/5], Train Loss: 0.5868
Epoch [5/5], Val Accuracy: 74.36%
--- Finetuning completado ---
✅ Resultado Final - Test Accuracy: 73.08%


# Prueba con Resnet18 para comparar

In [4]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
import torch.optim as optim

# Importa las librerías de MedMNIST
import medmnist
from medmnist import INFO  
import torchvision.transforms as transforms 

# --- 1. Configuración ---
SSL_WEIGHTS_PATH = "models/221025MG_backbone.ssl.pth"
DATASET_NAME = 'breastmnist'
BATCH_SIZE = 128                          
NUM_EPOCHS = 5                            
LEARNING_RATE = 1e-4                      # LR bajo es bueno para finetuning
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Usando dispositivo: {DEVICE}")

# --- 2. Definir el Modelo de Clasificación ---
# Este modelo combina el backbone + una cabeza para clasificar

class SslClassifier(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()
        self.backbone = backbone
        # La salida de ResNet18 (antes de la última capa) es 512
        self.fc = nn.Linear(512, num_classes) # Esta es la nueva cabeza

    def forward(self, x):
        # Pasa por el backbone (que ya sabe de SSL)
        features = self.backbone(x).flatten(start_dim=1)
        # Pasa por la cabeza de clasificación
        output = self.fc(features)
        return output

# --- 3. Cargar Datos (BreastMNIST) ---
info = INFO[DATASET_NAME]
num_classes = len(info['label'])
DataClass = getattr(medmnist, info['python_class'])


mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

data_transform = transforms.Compose([
    # Convierte la imagen (de 1 canal) a 3 canales duplicando el canal.
    transforms.Grayscale(num_output_channels=3),
    
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std) 
])

# Usamos el mismo transform para todos
train_transform = data_transform
val_transform = data_transform
test_transform = data_transform

# Crear los datasets
train_dataset = DataClass(split='train', transform=train_transform, download=True)
val_dataset = DataClass(split='val', transform=val_transform, download=True)
test_dataset = DataClass(split='test', transform=test_transform, download=True)

# Crear los DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Datos de {DATASET_NAME} cargados. Clases: {num_classes}")


# --- 4. Cargar Modelo (Versión ImageNet) ---

# 1. Carga la arquitectura ResNet18 CON pesos pre-entrenados de ImageNet
print("Cargando ResNet18 pre-entrenado en ImageNet...")
try:
    # Método nuevo (recomendado)
    resnet_imagenet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
except AttributeError:
    # Método antiguo (por si tu torchvision es viejo como medmnist)
    print("...usando fallback 'pretrained=True' por versión de torchvision.")
    resnet_imagenet = models.resnet18(pretrained=True)

# 2. Crea el backbone (quitando la capa final de 1000 clases de ImageNet)
backbone_imagenet = nn.Sequential(*list(resnet_imagenet.children())[:-1])
print("¡Backbone de ImageNet cargado!")

# 3. Crea el modelo de clasificación completo
#    (Re-usamos tu misma clase SslClassifier, que le añade la nueva cabeza)
model = SslClassifier(backbone_imagenet, num_classes).to(DEVICE)

# --- 5. Configurar Entrenamiento (Finetuning) ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print(f"--- Iniciando Finetuning (5 épocas) en {DATASET_NAME} ---")

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        labels = labels.squeeze().long()
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Train Loss: {running_loss / len(train_loader):.4f}")

    # Validación después de cada época
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            labels = labels.squeeze().long()
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    val_acc = 100 * val_correct / val_total
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}], Val Accuracy: {val_acc:.2f}%")

print("--- Finetuning completado ---")

# --- 6. Prueba Final (Clasificación) ---
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.squeeze().long()
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc = 100 * test_correct / test_total
print(f"==================================================")
print(f"✅ Resultado Final - Test Accuracy: {test_acc:.2f}%")
print(f"==================================================")

Usando dispositivo: cuda
Datos de breastmnist cargados. Clases: 2
Cargando ResNet18 pre-entrenado en ImageNet...
¡Backbone de ImageNet cargado!
--- Iniciando Finetuning (5 épocas) en breastmnist ---
Epoch [1/5], Train Loss: 0.7136
Epoch [1/5], Val Accuracy: 73.08%
Epoch [2/5], Train Loss: 0.3588
Epoch [2/5], Val Accuracy: 78.21%
Epoch [3/5], Train Loss: 0.2022
Epoch [3/5], Val Accuracy: 84.62%
Epoch [4/5], Train Loss: 0.1552
Epoch [4/5], Val Accuracy: 80.77%
Epoch [5/5], Train Loss: 0.0831
Epoch [5/5], Val Accuracy: 78.21%
--- Finetuning completado ---
✅ Resultado Final - Test Accuracy: 81.41%


# SSL sin finetuning

In [63]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
import torch.optim as optim

# Importa las librerías de MedMNIST
import medmnist
from medmnist import INFO  
import torchvision.transforms as transforms 

# --- Imports para fijar la semilla ---
import numpy as np
import random
import os

# --- INICIO: Bloque para Fijar Semillas ---

SEED = 52  # El mismo número para todas tus pruebas

def fijar_semillas(seed):
    # Para Python nativo
    random.seed(seed)
    # Para NumPy
    np.random.seed(seed)
    # Para PyTorch (CPU)
    torch.manual_seed(seed)
    
    # Para PyTorch (GPU)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # Si usas multi-GPU
        
        # Esto asegura la reproducibilidad en CUDA
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    print(f"Semillas fijadas en {seed}")

# --- ¡Llama a la función! ---
fijar_semillas(SEED)

# --- FIN: Bloque para Fijar Semillas ---


# --- 1. Configuración ---
DATASET_NAME = 'breastmnist'
BATCH_SIZE = 128                        
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Usando dispositivo: {DEVICE}")
print("--- MODO: Pura Inferencia (Sin Épocas) ---")

# --- 2. Definir el Modelo de Clasificación (Sin Cambios) ---
class SslClassifier(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()
        self.backbone = backbone
        self.fc = nn.Linear(512, num_classes) # ¡Esta cabeza se inicializará siempre igual!

    def forward(self, x):
        features = self.backbone(x).flatten(start_dim=1)
        output = self.fc(features)
        return output

# --- 3. Cargar Datos (BreastMNIST) (Sin Cambios) ---
info = INFO[DATASET_NAME]
num_classes = len(info['label'])
DataClass = getattr(medmnist, info['python_class'])

mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

data_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std) 
])

# Solo necesitamos el dataset de PRUEBA
test_transform = data_transform
test_dataset = DataClass(split='test', transform=test_transform, download=True)

# No necesitamos el 'worker_init_fn' aquí porque shuffle=False
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Datos de {DATASET_NAME} cargados. Clases: {num_classes}")

# --- 4. Cargar Modelo (Versión TU SSL) ---

SSL_WEIGHTS_PATH = "models/221025MG_backbone.ssl.pth" 

# 1. Crea la arquitectura del backbone
resnet = models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])

# 2. Carga TUS pesos SSL
backbone.load_state_dict(torch.load(SSL_WEIGHTS_PATH, map_location=DEVICE))
print("¡Pesos del backbone SSL cargados exitosamente!")

# 3. Crea el modelo de clasificación completo
model = SslClassifier(backbone, num_classes).to(DEVICE)

# --- 5. OMITIMOS EL ENTRENAMIENTO ---
# No hay bucle de épocas. Pasamos directo a la prueba.

# --- 6. Prueba Final (Clasificación) ---
print("--- Iniciando Prueba de Inferencia Pura ---")
model.eval() # Poner el modelo en modo evaluación
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.squeeze().long()
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc = 100 * test_correct / test_total
print(f"==================================================")
# --- Actualizamos el print final ---
print(f"✅ Resultado (Pura Inferencia, TU SSL) - Test Accuracy: {test_acc:.2f}%")
print(f"==================================================")

Semillas fijadas en 52
Usando dispositivo: cuda
--- MODO: Pura Inferencia (Sin Épocas) ---
Datos de breastmnist cargados. Clases: 2
¡Pesos del backbone SSL cargados exitosamente!
--- Iniciando Prueba de Inferencia Pura ---
✅ Resultado (Pura Inferencia, TU SSL) - Test Accuracy: 71.79%


# Resnet sin finetuning para comparar

In [66]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
import torch.optim as optim

# Importa las librerías de MedMNIST
import medmnist
from medmnist import INFO  
import torchvision.transforms as transforms 

# --- Imports para fijar la semilla ---
import numpy as np
import random
import os

# --- INICIO: Bloque para Fijar Semillas ---

SEED = 52  # El mismo número para todas tus pruebas

def fijar_semillas(seed):
    # Para Python nativo
    random.seed(seed)
    # Para NumPy
    np.random.seed(seed)
    # Para PyTorch (CPU)
    torch.manual_seed(seed)
    
    # Para PyTorch (GPU)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # Si usas multi-GPU
        
        # Esto asegura la reproducibilidad en CUDA
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    print(f"Semillas fijadas en {seed}")

# --- ¡Llama a la función! ---
fijar_semillas(SEED)

# --- FIN: Bloque para Fijar Semillas ---


# --- 1. Configuración ---
DATASET_NAME = 'breastmnist'
BATCH_SIZE = 128                        
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Usando dispositivo: {DEVICE}")
print("--- MODO: Pura Inferencia (Sin Épocas) ---")

# --- 2. Definir el Modelo de Clasificación ---
class SslClassifier(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()
        self.backbone = backbone
        self.fc = nn.Linear(512, num_classes) # ¡Esta cabeza se inicializará siempre igual!

    def forward(self, x):
        features = self.backbone(x).flatten(start_dim=1)
        output = self.fc(features)
        return output

# --- 3. Cargar Datos (BreastMNIST) (Sin Cambios) ---
info = INFO[DATASET_NAME]
num_classes = len(info['label'])
DataClass = getattr(medmnist, info['python_class'])

mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

data_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std) 
])

# Solo necesitamos el dataset de PRUEBA
test_transform = data_transform
test_dataset = DataClass(split='test', transform=test_transform, download=True)

# No necesitamos el 'worker_init_fn' aquí porque shuffle=False
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Datos de {DATASET_NAME} cargados. Clases: {num_classes}")

# --- 4. Cargar Modelo (Versión ImageNet) ---

print("Cargando ResNet18 pre-entrenado en ImageNet...")
try:
    resnet_imagenet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
except AttributeError:
    print("...usando fallback 'pretrained=True' por versión de torchvision.")
    resnet_imagenet = models.resnet18(pretrained=True)

backbone_imagenet = nn.Sequential(*list(resnet_imagenet.children())[:-1])
print("¡Backbone de ImageNet cargado!")

model = SslClassifier(backbone_imagenet, num_classes).to(DEVICE)

# --- 5. OMITIMOS EL ENTRENAMIENTO ---
# No hay bucle de épocas. Pasamos directo a la prueba.

# --- 6. Prueba Final (Clasificación) ---
print("--- Iniciando Prueba de Inferencia Pura ---")
model.eval() # Poner el modelo en modo evaluación
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.squeeze().long()
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc = 100 * test_correct / test_total
print(f"==================================================")
# --- Actualizamos el print final ---
print(f"✅ Resultado (Pura Inferencia, ImageNet) - Test Accuracy: {test_acc:.2f}%")
print(f"==================================================")

Semillas fijadas en 52
Usando dispositivo: cuda
--- MODO: Pura Inferencia (Sin Épocas) ---
Datos de breastmnist cargados. Clases: 2
Cargando ResNet18 pre-entrenado en ImageNet...
¡Backbone de ImageNet cargado!
--- Iniciando Prueba de Inferencia Pura ---
✅ Resultado (Pura Inferencia, ImageNet) - Test Accuracy: 41.67%


# Prueba con Resnet50

In [65]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
import torch.optim as optim

# Importa las librerías de MedMNIST
import medmnist
from medmnist import INFO  
import torchvision.transforms as transforms 

# --- Imports para fijar la semilla ---
import numpy as np
import random
import os

# --- INICIO: Bloque para Fijar Semillas ---

SEED = 52  # El mismo número para todas tus pruebas

def fijar_semillas(seed):
    # Para Python nativo
    random.seed(seed)
    # Para NumPy
    np.random.seed(seed)
    # Para PyTorch (CPU)
    torch.manual_seed(seed)
    
    # Para PyTorch (GPU)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # Si usas multi-GPU
        
        # Esto asegura la reproducibilidad en CUDA
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    print(f"Semillas fijadas en {seed}")

# --- ¡Llama a la función! ---
fijar_semillas(SEED)

# --- FIN: Bloque para Fijar Semillas ---


# --- 1. Configuración ---
DATASET_NAME = 'breastmnist'
BATCH_SIZE = 128                        
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Usando dispositivo: {DEVICE}")
print("--- MODO: Pura Inferencia (Sin Épocas) ---")

# --- 2. Definir el Modelo de Clasificación (para ResNet-50) ---
class SslClassifier(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()
        self.backbone = backbone
        # La salida de ResNet-50 es 2048
        self.fc = nn.Linear(2048, num_classes) # ¡Esta cabeza se inicializará siempre igual!

    def forward(self, x):
        features = self.backbone(x).flatten(start_dim=1)
        output = self.fc(features)
        return output

# --- 3. Cargar Datos (BreastMNIST) (Sin Cambios) ---
info = INFO[DATASET_NAME]
num_classes = len(info['label'])
DataClass = getattr(medmnist, info['python_class'])

mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

data_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std) 
])

# Solo necesitamos el dataset de PRUEBA
test_transform = data_transform
test_dataset = DataClass(split='test', transform=test_transform, download=True)

# No necesitamos el 'worker_init_fn' aquí porque shuffle=False
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Datos de {DATASET_NAME} cargados. Clases: {num_classes}")

# --- 4. Cargar Modelo (Versión ImageNet, ResNet-50) ---

print("Cargando ResNet-50 pre-entrenado en ImageNet...")
try:
    resnet_imagenet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
except AttributeError:
    print("...usando fallback 'pretrained=True' por versión de torchvision.")
    resnet_imagenet = models.resnet50(pretrained=True)

backbone_imagenet = nn.Sequential(*list(resnet_imagenet.children())[:-1])
print("¡Backbone de ResNet-50 (ImageNet) cargado!")

model = SslClassifier(backbone_imagenet, num_classes).to(DEVICE)

# --- 5. OMITIMOS EL ENTRENAMIENTO ---
# No hay bucle de épocas. Pasamos directo a la prueba.

# --- 6. Prueba Final (Clasificación) ---
print("--- Iniciando Prueba de Inferencia Pura ---")
model.eval() # Poner el modelo en modo evaluación
test_correct = 0
test_total = 0

with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.squeeze().long()
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_acc = 100 * test_correct / test_total
print(f"==================================================")
# --- Actualizamos el print final ---
print(f"✅ Resultado (Pura Inferencia, ResNet-50) - Test Accuracy: {test_acc:.2f}%")
print(f"==================================================")

Semillas fijadas en 52
Usando dispositivo: cuda
--- MODO: Pura Inferencia (Sin Épocas) ---
Datos de breastmnist cargados. Clases: 2
Cargando ResNet-50 pre-entrenado en ImageNet...
¡Backbone de ResNet-50 (ImageNet) cargado!
--- Iniciando Prueba de Inferencia Pura ---
✅ Resultado (Pura Inferencia, ResNet-50) - Test Accuracy: 46.15%


# Lo mismo pero con chestmnist

In [64]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
import torch.optim as optim

# Importa las librerías de MedMNIST
import medmnist
from medmnist import INFO  
import torchvision.transforms as transforms 

# --- Imports para fijar la semilla ---
import numpy as np
import random
import os

# --- INICIO: Bloque para Fijar Semillas ---

SEED = 52  # El mismo número para todas tus pruebas

def fijar_semillas(seed):
    # Para Python nativo
    random.seed(seed)
    # Para NumPy
    np.random.seed(seed)
    # Para PyTorch (CPU)
    torch.manual_seed(seed)
    
    # Para PyTorch (GPU)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # Si usas multi-GPU
        
        # Esto asegura la reproducibilidad en CUDA
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    print(f"Semillas fijadas en {seed}")

# --- ¡Llama a la función! ---
fijar_semillas(SEED)

# --- FIN: Bloque para Fijar Semillas ---


# --- 1. Configuración ---
DATASET_NAME = 'chestmnist'
BATCH_SIZE = 128                        
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Usando dispositivo: {DEVICE}")
print("--- MODO: Pura Inferencia (Sin Épocas) ---")

# --- 2. Definir el Modelo de Clasificación (Sin Cambios) ---
class SslClassifier(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()
        self.backbone = backbone
        self.fc = nn.Linear(512, num_classes) # ¡Esta cabeza se inicializará siempre igual!

    def forward(self, x):
        features = self.backbone(x).flatten(start_dim=1)
        output = self.fc(features)
        return output

# --- 3. Cargar Datos (BreastMNIST) (Sin Cambios) ---
info = INFO[DATASET_NAME]
num_classes = len(info['label'])
DataClass = getattr(medmnist, info['python_class'])

mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

data_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std) 
])

# Solo necesitamos el dataset de PRUEBA
test_transform = data_transform
test_dataset = DataClass(split='test', transform=test_transform, download=True)

# No necesitamos el 'worker_init_fn' aquí porque shuffle=False
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Datos de {DATASET_NAME} cargados. Clases: {num_classes}")

# --- 4. Cargar Modelo (Versión TU SSL) ---

SSL_WEIGHTS_PATH = "models/221025MG_backbone.ssl.pth" 

# 1. Crea la arquitectura del backbone
resnet = models.resnet18()
backbone = nn.Sequential(*list(resnet.children())[:-1])

# 2. Carga TUS pesos SSL
backbone.load_state_dict(torch.load(SSL_WEIGHTS_PATH, map_location=DEVICE))
print("¡Pesos del backbone SSL cargados exitosamente!")

# 3. Crea el modelo de clasificación completo
model = SslClassifier(backbone, num_classes).to(DEVICE)

# --- 5. OMITIMOS EL ENTRENAMIENTO ---
# No hay bucle de épocas. Pasamos directo a la prueba.

# --- 6. Prueba Final (Clasificación - LÓGICA MULTI-ETIQUETA) ---
print("--- Iniciando Prueba de Inferencia Pura ---")
model.eval() # Poner el modelo en modo evaluación
test_correct_labels = 0
test_total_labels = 0

with torch.no_grad():
    for images, labels in test_loader:
        # labels tiene shape [batch_size, 14] y es float
        
        # --- NO USAMOS SQUEEZE ---
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        outputs = model(images) # shape [batch_size, 14]
        
        # --- NUEVA LÓGICA DE PREDICCIÓN (MULTI-ETIQUETA) ---
        
        # 1. Aplicar Sigmoid a las salidas (logits) para obtener probabilidades [0, 1]
        probs = torch.sigmoid(outputs)
        
        # 2. Obtener predicciones (0 o 1) usando un umbral de 0.5
        predicted = (probs > 0.5).float() # Shape [batch_size, 14]
        
        # 3. Calcular aciertos
        # Comparamos las predicciones (0/1) con las etiquetas (0.0/1.0)
        test_correct_labels += (predicted == labels.float()).sum().item()
        
        # 4. Contar el número total de etiquetas evaluadas
        test_total_labels += labels.numel() # numel() es batch_size * 14
        
# --- FIN DEL BUCLE ---

# Calcular el accuracy total de etiquetas
test_acc = 100 * test_correct_labels / test_total_labels

print(f"==================================================")
# --- Actualizamos el print final ---
print(f"✅ Resultado (Pura Inferencia, TU SSL) - Test Accuracy: {test_acc:.2f}%")
print(f"==================================================")

Semillas fijadas en 52
Usando dispositivo: cuda
--- MODO: Pura Inferencia (Sin Épocas) ---
Datos de chestmnist cargados. Clases: 14
¡Pesos del backbone SSL cargados exitosamente!
--- Iniciando Prueba de Inferencia Pura ---
✅ Resultado (Pura Inferencia, TU SSL) - Test Accuracy: 47.80%


In [67]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
import torch.optim as optim

# Importa las librerías de MedMNIST
import medmnist
from medmnist import INFO  
import torchvision.transforms as transforms 

# --- Imports para fijar la semilla ---
import numpy as np
import random
import os

# --- INICIO: Bloque para Fijar Semillas ---

SEED = 52  # El mismo número para todas tus pruebas

def fijar_semillas(seed):
    # Para Python nativo
    random.seed(seed)
    # Para NumPy
    np.random.seed(seed)
    # Para PyTorch (CPU)
    torch.manual_seed(seed)
    
    # Para PyTorch (GPU)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # Si usas multi-GPU
        
        # Esto asegura la reproducibilidad en CUDA
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    print(f"Semillas fijadas en {seed}")

# --- ¡Llama a la función! ---
fijar_semillas(SEED)

# --- FIN: Bloque para Fijar Semillas ---


# --- 1. Configuración ---
DATASET_NAME = 'chestmnist'
BATCH_SIZE = 128                        
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Usando dispositivo: {DEVICE}")
print("--- MODO: Pura Inferencia (Sin Épocas) ---")

# --- 2. Definir el Modelo de Clasificación (Sin Cambios) ---
class SslClassifier(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()
        self.backbone = backbone
        self.fc = nn.Linear(512, num_classes) # ¡Esta cabeza se inicializará siempre igual!

    def forward(self, x):
        features = self.backbone(x).flatten(start_dim=1)
        output = self.fc(features)
        return output

# --- 3. Cargar Datos (ChestMNIST) ---
info = INFO[DATASET_NAME]
num_classes = len(info['label']) # Esto será 14
DataClass = getattr(medmnist, info['python_class'])

mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

data_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std) 
])

# Solo necesitamos el dataset de PRUEBA
test_transform = data_transform
test_dataset = DataClass(split='test', transform=test_transform, download=True)

# No necesitamos el 'worker_init_fn' aquí porque shuffle=False
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Datos de {DATASET_NAME} cargados. Clases: {num_classes}")

# --- 4. Cargar Modelo (Versión ImageNet) ---

# --- ¡¡CAMBIO AQUÍ!! ---
print("Cargando ResNet18 pre-entrenado en ImageNet...")
try:
    resnet_imagenet = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
except AttributeError:
    print("...usando fallback 'pretrained=True' por versión de torchvision.")
    resnet_imagenet = models.resnet18(pretrained=True)

backbone_imagenet = nn.Sequential(*list(resnet_imagenet.children())[:-1])
print("¡Backbone de ImageNet cargado!")
# --- FIN DEL CAMBIO ---

# 3. Crea el modelo de clasificación completo
model = SslClassifier(backbone_imagenet, num_classes).to(DEVICE)

# --- 5. OMITIMOS EL ENTRENAMIENTO ---
# No hay bucle de épocas. Pasamos directo a la prueba.

# --- 6. Prueba Final (Clasificación - LÓGICA MULTI-ETIQUETA) ---
print("--- Iniciando Prueba de Inferencia Pura ---")
model.eval() # Poner el modelo en modo evaluación
test_correct_labels = 0
test_total_labels = 0

with torch.no_grad():
    for images, labels in test_loader:
        # labels tiene shape [batch_size, 14] y es float
        
        # --- NO USAMOS SQUEEZE ---
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        outputs = model(images) # shape [batch_size, 14]
        
        # --- NUEVA LÓGICA DE PREDICCIÓN (MULTI-ETIQUETA) ---
        
        # 1. Aplicar Sigmoid a las salidas (logits) para obtener probabilidades [0, 1]
        probs = torch.sigmoid(outputs)
        
        # 2. Obtener predicciones (0 o 1) usando un umbral de 0.5
        predicted = (probs > 0.5).float() # Shape [batch_size, 14]
        
        # 3. Calcular aciertos
        # Comparamos las predicciones (0/1) con las etiquetas (0.0/1.0)
        test_correct_labels += (predicted == labels.float()).sum().item()
        
        # 4. Contar el número total de etiquetas evaluadas
        test_total_labels += labels.numel() # numel() es batch_size * 14
        
# --- FIN DEL BUCLE ---

# Calcular el accuracy total de etiquetas
test_acc = 100 * test_correct_labels / test_total_labels

print(f"==================================================")
# --- Actualizamos el print final ---
print(f"✅ Resultado (Pura Inferencia, ImageNet) - Test Accuracy: {test_acc:.2f}%")
print(f"==================================================")

Semillas fijadas en 52
Usando dispositivo: cuda
--- MODO: Pura Inferencia (Sin Épocas) ---
Datos de chestmnist cargados. Clases: 14
Cargando ResNet18 pre-entrenado en ImageNet...
¡Backbone de ImageNet cargado!
--- Iniciando Prueba de Inferencia Pura ---
✅ Resultado (Pura Inferencia, ImageNet) - Test Accuracy: 61.30%


In [68]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader
import torch.optim as optim

# Importa las librerías de MedMNIST
import medmnist
from medmnist import INFO  
import torchvision.transforms as transforms 

# --- Imports para fijar la semilla ---
import numpy as np
import random
import os

# --- INICIO: Bloque para Fijar Semillas ---

SEED = 52  # El mismo número para todas tus pruebas

def fijar_semillas(seed):
    # Para Python nativo
    random.seed(seed)
    # Para NumPy
    np.random.seed(seed)
    # Para PyTorch (CPU)
    torch.manual_seed(seed)
    
    # Para PyTorch (GPU)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # Si usas multi-GPU
        
        # Esto asegura la reproducibilidad en CUDA
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    print(f"Semillas fijadas en {seed}")

# --- ¡Llama a la función! ---
fijar_semillas(SEED)

# --- FIN: Bloque para Fijar Semillas ---


# --- 1. Configuración ---
DATASET_NAME = 'chestmnist'
BATCH_SIZE = 128                        
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Usando dispositivo: {DEVICE}")
print("--- MODO: Pura Inferencia (Sin Épocas) ---")

# --- 2. Definir el Modelo de Clasificación ---
# --- ¡¡CAMBIO AQUÍ para ResNet-50!! ---
class SslClassifier(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()
        self.backbone = backbone
        # La salida de ResNet-50 es 2048, no 512
        self.fc = nn.Linear(2048, num_classes) # ¡Esta cabeza se inicializará siempre igual!

    def forward(self, x):
        features = self.backbone(x).flatten(start_dim=1)
        output = self.fc(features)
        return output

# --- 3. Cargar Datos (ChestMNIST) ---
info = INFO[DATASET_NAME]
num_classes = len(info['label']) # Esto será 14
DataClass = getattr(medmnist, info['python_class'])

mean = [0.5, 0.5, 0.5]
std = [0.5, 0.5, 0.5]

data_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std) 
])

# Solo necesitamos el dataset de PRUEBA
test_transform = data_transform
test_dataset = DataClass(split='test', transform=test_transform, download=True)

# No necesitamos el 'worker_init_fn' aquí porque shuffle=False
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"Datos de {DATASET_NAME} cargados. Clases: {num_classes}")

# --- 4. Cargar Modelo (Versión ImageNet, ResNet-50) ---

# --- ¡¡CAMBIO AQUÍ para ResNet-50!! ---
print("Cargando ResNet-50 pre-entrenado en ImageNet...")
try:
    # Cambiamos resnet18 por resnet50
    resnet_imagenet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2) # V2 es la más reciente
except AttributeError:
    print("...usando fallback 'pretrained=True' por versión de torchvision.")
    resnet_imagenet = models.resnet50(pretrained=True)

backbone_imagenet = nn.Sequential(*list(resnet_imagenet.children())[:-1])
print("¡Backbone de ResNet-50 (ImageNet) cargado!")
# --- FIN DEL CAMBIO ---

# 3. Crea el modelo de clasificación completo
model = SslClassifier(backbone_imagenet, num_classes).to(DEVICE)

# --- 5. OMITIMOS EL ENTRENAMIENTO ---
# No hay bucle de épocas. Pasamos directo a la prueba.

# --- 6. Prueba Final (Clasificación - LÓGICA MULTI-ETIQUETA) ---
print("--- Iniciando Prueba de Inferencia Pura ---")
model.eval() # Poner el modelo en modo evaluación
test_correct_labels = 0
test_total_labels = 0

with torch.no_grad():
    for images, labels in test_loader:
        # labels tiene shape [batch_size, 14] y es float
        
        # --- NO USAMOS SQUEEZE ---
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        outputs = model(images) # shape [batch_size, 14]
        
        # --- NUEVA LÓGICA DE PREDICCIÓN (MULTI-ETIQUETA) ---
        
        # 1. Aplicar Sigmoid a las salidas (logits) para obtener probabilidades [0, 1]
        probs = torch.sigmoid(outputs)
        
        # 2. Obtener predicciones (0 o 1) usando un umbral de 0.5
        predicted = (probs > 0.5).float() # Shape [batch_size, 14]
        
        # 3. Calcular aciertos
        # Comparamos las predicciones (0/1) con las etiquetas (0.0/1.0)
        test_correct_labels += (predicted == labels.float()).sum().item()
        
        # 4. Contar el número total de etiquetas evaluadas
        test_total_labels += labels.numel() # numel() es batch_size * 14
        
# --- FIN DEL BUCLE ---

# Calcular el accuracy total de etiquetas
test_acc = 100 * test_correct_labels / test_total_labels

print(f"==================================================")
# --- Actualizamos el print final ---
print(f"✅ Resultado (Pura Inferencia, ResNet-50) - Test Accuracy: {test_acc:.2f}%")
print(f"==================================================")

Semillas fijadas en 52
Usando dispositivo: cuda
--- MODO: Pura Inferencia (Sin Épocas) ---
Datos de chestmnist cargados. Clases: 14
Cargando ResNet-50 pre-entrenado en ImageNet...
¡Backbone de ResNet-50 (ImageNet) cargado!
--- Iniciando Prueba de Inferencia Pura ---
✅ Resultado (Pura Inferencia, ResNet-50) - Test Accuracy: 42.59%
